In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
                      
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import metrics
from sklearn import linear_model
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingClassifier

import nltk
from nltk.stem import WordNetLemmatizer

from sklearn.preprocessing import Normalizer


from sklearn.model_selection import RandomizedSearchCV
import time

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# path = r"/Users/charlotte/Desktop/BT4222/train_labelled_final.csv"
# reviews = pd.read_csv(path)
reviews = pd.read_csv('./train_labelled.csv', encoding= 'unicode_escape')

In [4]:
reviews.head()

,Restaurant,Review,Label,Stars,Date
0,Happy Tummy,"Fresh ingredients, friendly peeps and so much ...",1,5,2016-04-06T00:00:00
1,Cibo Italiano,A small selection of Italian wines by the glas...,1,4,2015-12-24T00:00:00
2,Yan kee Noodle House,The plus point is that the price remains the s...,1,4,2018-12-28T00:00:00
3,Clinton Street Baking Company & Restaurant,Same for more?I ordered what I thought was the...,1,2,2018-03-03T00:00:00
4,Song Fa Bak Kut Teh,I will probably get that again.Have been very ...,5,5,2019-01-05T00:00:00


In [5]:
reviews.Label.value_counts()

2    916
5    645
3    525
1    394
4    350
Name: Label, dtype: int64

In [6]:
# Cleaning the Data
def clean(data):

    # Removing leading and trailing white spaces
    data = str(data).strip()
    
    # Converting all text to lower case
    data = data.lower() 
    
    # add space for punctuation
    translator = re.compile('[%s]' % re.escape(string.punctuation))
    translator.sub(' ', data)

    #remove punct
    data = re.sub('[^A-Za-z0-9]+', ' ', data)
    
    #keep english words
    data = re.sub(r'[^\x00-\x7F]+', '', data)
    
    #keep printable
    data = re.sub(f'[^{re.escape(string.printable)}]', '', data)

    # Removing Stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(data)

    # Converting all text to base form
#     lemmatizer = WordNetLemmatizer()
#     tokens_lemmatized = [lemmatizer.lemmatize(token) for token in tokens]
            
    # Removing Punctuation
#     translator = str.maketrans('', '', string.punctuation)
#     data = data.translate(translator)

         # Converting all numbers to words
#    word = data.split(' ')
#     p = inflect.engine()
#     word = ' '.join([i for i in word if not i.isdigit()])
#     word = re.sub(r'\d+', p.number_to_words(word) , word)    

    word = ' '.join([i for i in tokens if not i in stop_words])

    return word

# clean text data
reviews["Review_clean"] = reviews["Review"].apply(lambda x: clean(x))
reviews.head()

,Restaurant,Review,Label,Stars,Date,Review_clean
0,Happy Tummy,"Fresh ingredients, friendly peeps and so much ...",1,5,2016-04-06T00:00:00,fresh ingredients friendly peeps much cheaper ...
1,Cibo Italiano,A small selection of Italian wines by the glas...,1,4,2015-12-24T00:00:00,small selection italian wines glass beers well...
2,Yan kee Noodle House,The plus point is that the price remains the s...,1,4,2018-12-28T00:00:00,plus point price remains 4 per bowl
3,Clinton Street Baking Company & Restaurant,Same for more?I ordered what I thought was the...,1,2,2018-03-03T00:00:00,ordered thought signature omelette name
4,Song Fa Bak Kut Teh,I will probably get that again.Have been very ...,5,5,2019-01-05T00:00:00,probably get tempted pick soup base sell sure ...


In [7]:
reviews.Review_clean

0       fresh ingredients friendly peeps much cheaper ...
1       small selection italian wines glass beers well...
2                     plus point price remains 4 per bowl
3                 ordered thought signature omelette name
4       probably get tempted pick soup base sell sure ...
                              ...                        
2825                                   went weekday night
2826                      came birthday celebration mates
2827                                   caters vegetarians
2828    friend really wanted come back try foods unfor...
2829                          issue place drive take taxi
Name: Review_clean, Length: 2830, dtype: object

In [149]:
# def clean(data):
    
#     # Removing leading and trailing white spaces
# #     data = str(data).strip()

# # add space for punctuation
#     translator = re.compile('[%s]' % re.escape(string.punctuation))
#     translator.sub(' ', data)
#     # Converting all text to lower case
#     data = data.lower() 
    
#     #remove punct
#     data = re.sub('[^A-Za-z0-9]+', ' ', data)
    
#     #keep english words
#     data = re.sub(r'[^\x00-\x7F]+', '', data)
#     #keep printable
#     data = re.sub(f'[^{re.escape(string.printable)}]', '', data)
    
#     # Converting all numbers to words
# #     word = data.split(' ')
# #     p = inflect.engine()
# #     word = ' '.join([i for i in word if not i.isdigit()])
# #     word = re.sub(r'\d+', p.number_to_words(word) , word) 

# #     translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
# #     # Removing Punctuation
# #     #word = data.split(' ')
# #     translator = str.maketrans('', '', string.punctuation)
# #     data = data.translate(translator)

#     # Removing Stopwords
#     stop_words = set(stopwords.words('english'))
#     tokens = word_tokenize(data)
#     data = ' '.join([i for i in tokens if not i in stop_words])
    
#     # Converting all text to base form
#     lemmatizer = WordNetLemmatizer()
#     tokens_lemmatized = [lemmatizer.lemmatize(token) for token in tokens]

#     return data

# # clean text data
# df['review_clean'] = df['Review'].apply(lambda x: clean(x))
# df.head()



In [8]:
x = reviews.Review_clean
y = reviews.Label

In [9]:
# split x and y into training and testing sets
# stratify returns training and test subsets that have the same proportions of class labels as the input dataset.
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, stratify = y)

In [10]:
# examine the object shapes
print(y_train.value_counts())
print(y_test.value_counts())

2    687
5    484
3    394
1    295
4    262
Name: Label, dtype: int64
2    229
5    161
3    131
1     99
4     88
Name: Label, dtype: int64


In [ ]:
## Building the vocabulary

In [211]:
vect = CountVectorizer()

In [212]:
vect.fit(x_train)
x_train_dtm = vect.transform(x_train)

# only transform x_test
x_test_dtm = vect.transform(x_test)

In [213]:
# examine the shapes: rows are documents, columns are terms (aka "tokens" or "features")
print(x_train_dtm.shape)
print(x_test_dtm.shape)

(1444, 3695)
(482, 3695)


In [214]:
# examine the last 50 features
print(vect.get_feature_names()[-50:])

['workers', 'working', 'workout', 'workplace', 'works', 'world', 'worse', 'worst', 'worth', 'worthy', 'would', 'wow', 'wowed', 'wrap', 'wrapped', 'written', 'wrong', 'wrongly', 'xeo', 'xie', 'xlb', 'xo', 'yahoo', 'yakitori', 'yam', 'yamazaki', 'yang', 'year', 'years', 'yelp', 'yelper', 'yeoh', 'yes', 'yet', 'yo', 'yoghurt', 'yogurt', 'yolk', 'yong', 'yu', 'yuan', 'yuen', 'yummier', 'yummmmmmyyyy', 'yummmy', 'yummy', 'yuzu', 'zach', 'zealand', 'zucchini']


# Comparing the accuracy of different approaches

In [215]:
vects = [CountVectorizer(),TfidfVectorizer()]
vectnames = ["Count Vect","Tfidf Vect"]


clfs = [BaggingClassifier(RandomForestClassifier()), MultinomialNB(),LinearSVC(),LogisticRegression(),RandomForestClassifier(),linear_model.SGDClassifier(), GradientBoostingClassifier()]
clfnames = ["bagging random forest", "Naive Bayes","Linear SVM","Logistic Regression","Random Forest","Stochastic Gradient Descent", "Gradient Boosting"]

In [216]:
#building a pipeline

for vectname, vect, in zip(vectnames, vects):

    for clfname, clf in zip(clfnames, clfs):
        pipe = Pipeline([
            ('vect', vect),
            ('clf', clf),
        ]
        )       

        pipe.fit(x_train, y_train)
        pred = pipe.predict(x_test)
        train_acc = metrics.accuracy_score(y_train, pipe.predict(x_train))
        test_acc = metrics.accuracy_score(y_test, pred)
        #roc_auc = metrics.roc_auc_score(y_test, pred)
        print("{} + {} - train acc: {} test acc: {} ".format(vectname, clfname, train_acc, test_acc))
        #scores = cross_val_score(pipe, x, y,scoring='roc_auc')
        #print(scores.mean())

    

Count Vect + bagging random forest - train acc: 0.9300554016620498 test acc: 0.6203319502074689 
Count Vect + Naive Bayes - train acc: 0.9335180055401662 test acc: 0.6991701244813278 
Count Vect + Linear SVM - train acc: 0.997229916897507 test acc: 0.6721991701244814 
Count Vect + Logistic Regression - train acc: 0.9840720221606648 test acc: 0.6970954356846473 
Count Vect + Random Forest - train acc: 0.9847645429362881 test acc: 0.6307053941908713 
Count Vect + Stochastic Gradient Descent - train acc: 0.9986149584487535 test acc: 0.6784232365145229 
Count Vect + Gradient Boosting - train acc: 0.8795013850415513 test acc: 0.6721991701244814 
Tfidf Vect + bagging random forest - train acc: 0.9425207756232687 test acc: 0.6535269709543569 
Tfidf Vect + Naive Bayes - train acc: 0.871191135734072 test acc: 0.5871369294605809 
Tfidf Vect + Linear SVM - train acc: 0.9958448753462604 test acc: 0.7095435684647303 
Tfidf Vect + Logistic Regression - train acc: 0.9023545706371191 test acc: 0.67219

In [219]:
# doing randomised search 

from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('lr', LogisticRegression())
])


parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__analyzer':['char', 'word', 'char_wb'],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tfidf__binary': [True, False],
    'tfidf__norm': [None, 'l1', 'l2'], 
    'lr__dual': [True,False],
    'lr__max_iter': [100,200, 500],
    'lr__C' :np.logspace(0, 4, num=10)
  
    
}



random =RandomizedSearchCV(pipe, parameters, cv=5, n_jobs=2)

start_time = time.time()
random_result = random.fit(x_train, y_train)
# Summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))
print("Execution time: " + str((time.time() - start_time)) + ' ms')

pred = random.predict(x_test)
test_acc = metrics.accuracy_score(y_test, pred)
#roc_auc = metrics.roc_auc_score(y_test, pred)
pre_macro = metrics.precision_score(y_test, pred, average="macro")
recall_macro = metrics.recall_score(y_test, pred, average="macro")
f1_macro = metrics.f1_score(y_test, pred, average="macro")


print("test acc: {} recall: {} f1: {}".format(test_acc, recall_macro, f1_macro))


Best: 0.689751 using {'tfidf__norm': 'l1', 'tfidf__ngram_range': (1, 3), 'tfidf__max_df': 0.75, 'tfidf__binary': True, 'tfidf__analyzer': 'word', 'lr__max_iter': 200, 'lr__dual': True, 'lr__C': 59.94842503189409}
Execution time: 6.650973796844482 ms
test acc: 0.7012448132780082 recall: 0.6678931772663853 f1: 0.68085743258033


In [218]:
vect = TfidfVectorizer(ngram_range=(1, 2), max_df=0.75, norm= 'l2', binary = True, analyzer = 'word')
x_train_dtm = vect.fit_transform(x_train)
x_test_dtm = vect.transform(x_test)


lr = LogisticRegression(max_iter = 100, dual = False, C = 10000)
lr.fit(x_train_dtm, y_train)
y_pred_class = lr.predict(x_test_dtm)

# Get the training accuracy
print('Training Accuracy: ', metrics.accuracy_score(y_train, lr.predict(x_train_dtm)))
# print the accuracy of its predictions
print('Test Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))


Training Accuracy:  1.0
Test Accuracy:  0.7053941908713693


In [222]:
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC



vect = TfidfVectorizer(ngram_range=(1, 2), max_df=0.75, norm= 'l2', binary = True, analyzer = 'word')
x_train_dtm = vect.fit_transform(x_train)
x_test_dtm = vect.transform(x_test)


lr = OneVsRestClassifier(SVC())
lr.fit(x_train_dtm, y_train)
y_pred_class = lr.predict(x_test_dtm)

# Get the training accuracy
print('Training Accuracy: ', metrics.accuracy_score(y_train, lr.predict(x_train_dtm)))
# print the accuracy of its predictions
print('Test Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))

Training Accuracy:  0.9785318559556787
Test Accuracy:  0.6556016597510373


In [ ]:
## RNN

In [11]:
# Import necessary libraries
import keras
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional
from keras.models import Model
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from keras.models import Sequential

Using TensorFlow backend.


In [12]:
# Build input sequence for running the model
# max_len = 150
num_words = 1000
tok = Tokenizer(num_words= num_words)
tok.fit_on_texts(x_train)
word_idx = tok.word_index

In [13]:
# Transforming to matrix
trainmatrix = tok.texts_to_matrix(x_train)
testmatrix = tok.texts_to_matrix(x_test)

In [14]:
encoder = LabelEncoder()
encoder.fit(y_train)
trainlabels = encoder.transform(y_train)
testlabels = encoder.transform(y_test)

num_classes = np.max(y_train) + 1
num_classes

6

In [15]:
y_train = to_categorical(trainlabels, num_classes)
y_test = to_categorical(testlabels, num_classes)

In [16]:
model= Sequential()
model.add(Dense(512, input_shape=(num_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [17]:
# Build and compile the model
model.compile(loss='binary_crossentropy',optimizer= 'adam',metrics=['accuracy'])    
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               512512    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 3078      
_________________________________________________________________
activation_2 (Activation)    (None, 6)                 0         
Total params: 515,590
Trainable params: 515,590
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Train the model
model.fit(trainmatrix,y_train,validation_split = 0.2, 
           batch_size=32, epochs=5, 
          callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)]
         )

Train on 1697 samples, validate on 425 samples
Epoch 1/5
1697/1697 [==============================] - 2s 1ms/step - loss: 0.4031 - accuracy: 0.8361 - val_loss: 0.3585 - val_accuracy: 0.8510
Epoch 2/5
1697/1697 [==============================] - 1s 540us/step - loss: 0.2973 - accuracy: 0.8725 - val_loss: 0.2832 - val_accuracy: 0.8820
Epoch 3/5
1697/1697 [==============================] - 1s 457us/step - loss: 0.2019 - accuracy: 0.9187 - val_loss: 0.2384 - val_accuracy: 0.8969
Epoch 4/5
1697/1697 [==============================] - 1s 312us/step - loss: 0.1388 - accuracy: 0.9518 - val_loss: 0.2209 - val_accuracy: 0.9125
Epoch 5/5
1697/1697 [==============================] - 1s 480us/step - loss: 0.1045 - accuracy: 0.9672 - val_loss: 0.2246 - val_accuracy: 0.9110


In [19]:
# Evaluate results for testing data
accr = model.evaluate(testmatrix,y_test)

708/708 [==============================] - 0s 131us/step


In [20]:
# Print evaluation results
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.237
  Accuracy: 0.906
